# **StarGAN**<br/>
**Master's Degree in Data Science (A.Y. 2023/2024)**<br/>
**University of Milano - Bicocca**<br/>

Vittorio Haardt, Luca Porcelli

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil
import random
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf
from torch.utils import data
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from torchvision.utils import save_image
import time
import datetime
import argparse
from torch.backends import cudnn

## Train Data

Import the training data from the zipped file

In [ ]:
!unzip "/content/drive/MyDrive/Digital/Data/Imm.zip" -d Data

Remove the unwanted class

In [ ]:
cartella_principale = '/content/Data/dataset'

cartella_da_elimare = os.path.join(cartella_principale, 'Ahegao')
shutil.rmtree(cartella_da_elimare)

# List of allowed extension
estensioni_consentite = ['.jpg', '.jpeg', '.png']

# Controll if a file has a proibited extension
def is_estensione_consentita(file):
    return any(file.lower().endswith(ext) for ext in estensioni_consentite)

# Itera through the main folder and subfolders
for cartella_padre, _, files in os.walk(cartella_principale):
    for file in files:
        percorso_file = os.path.join(cartella_padre, file)
        if not is_estensione_consentita(file):
            os.remove(percorso_file)
            print(f"Eliminato il file: {percorso_file}")

Eliminato il file: /content/Data/dataset/Sad/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp
Eliminato il file: /content/Data/dataset/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp
Eliminato il file: /content/Data/dataset/Angry/bb807308129b1810a9789477d7587af2838cb6ebb6e6e724d59ae34b~angry.bmp
Eliminato il file: /content/Data/dataset/Neutral/0a20898ed0ae9f058112516f3bff1885f57e71188b148300caf23e37f.bmp
Eliminato il file: /content/Data/dataset/Neutral/0c44e659d8adc399e063533722f72fd8fe885905f1aa5f0e07928f64f.bmp


We resize all images to the same dimension and in the format of 256x256 so that during the generation process, no important parts of the faces are cut off.

In [ ]:
def make_square(image_path, output_size=(256, 256)):
    """
    The function renders a square image by centering and resizing it.
    :param image_path: The path to the original image.
    :param output_size: The desired size of the output square image.
    :return: The square image.
    """
    img = Image.open(image_path)
    img = img.resize(output_size, Image.ANTIALIAS)
    new_img = Image.new("RGB", output_size, (255, 255, 255))
    position = ((output_size[0] - img.size[0]) // 2, (output_size[1] - img.size[1]) // 2)
    new_img.paste(img, position)
    return new_img

def process_images_in_directory(input_dir, output_dir):
    """
    Processes all the images present in a folder and makes them square.
    :param input_dir: Input folder containing the images.
    :param output_dir: Output folder where the square images will be saved.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in tqdm(files):
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                input_path = os.path.join(root, file)
                output_subdir = os.path.join(output_dir, os.path.relpath(root, input_dir))
                output_path = os.path.join(output_subdir, file)
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)
                square_img = make_square(input_path)
                square_img.save(output_path)


In [ ]:
input_directory = "/content/Data/dataset"
output_directory = "/content/Data/dataset_quad"
process_images_in_directory(input_directory, output_directory)

0it [00:00, ?it/s]
  0%|          | 0/3933 [00:00<?, ?it/s]<ipython-input-5-2eab3d3d7807>:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(output_size, Image.ANTIALIAS)
100%|██████████| 4025/4025 [01:23<00:00, 48.23it/s]


## STAR GAN Functions

Residual block used in the models.

*   Discriminator
*   Generator

Discriminator and generator models that are used in the GAN architecture.

In [3]:
class ResidualBlock(nn.Module):
    """Residual Block with instance normalization."""
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim_out, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)


class Generator(nn.Module):
    """Generator network."""
    def __init__(self, conv_dim=64, c_dim=5, repeat_num=6):
        super(Generator, self).__init__()

        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True))

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(2):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))

        # Up-sampling layers.
        for i in range(2):
            layers.append(nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim // 2

        layers.append(nn.Conv2d(curr_dim, 3, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.Tanh())
        self.main = nn.Sequential(*layers)

    def forward(self, x, c):
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)
        return self.main(x)


class Discriminator(nn.Module):
    """Discriminator network with PatchGAN."""
    def __init__(self, image_size=128, conv_dim=64, c_dim=5, repeat_num=6):
        super(Discriminator, self).__init__()
        layers = []
        layers.append(nn.Conv2d(3, conv_dim, kernel_size=4, stride=2, padding=1))
        layers.append(nn.LeakyReLU(0.01))

        curr_dim = conv_dim
        for i in range(1, repeat_num):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1))
            layers.append(nn.LeakyReLU(0.01))
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num))
        self.main = nn.Sequential(*layers)
        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, bias=False)

    def forward(self, x):
        h = self.main(x)
        out_src = self.conv1(h)
        out_cls = self.conv2(h)
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))

Wrapper for logging data to TensorBoard, which is a visualization tool used for monitoring and understanding the behavior of TensorFlow models.

In [4]:
class Logger(object):
    """Tensorboard logger."""

    def __init__(self, log_dir):
        """Initialize summary writer."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Add scalar summary."""
        with self.writer.as_default():
            tf.summary.scalar(tag, value, step=step)
            self.writer.flush()

Data loader: The loader apply some trasformation to the imput image, and in the training some data agumentation.

In [5]:
def get_loader(image_dir, attr_path, selected_attrs, crop_size=178, image_size=224,
               batch_size=16, mode='train', num_workers=1):
    """Build and return a data loader."""
    transform = []
    if mode == 'train':
        transform.append(T.RandomHorizontalFlip())
    transform.append(T.CenterCrop(crop_size))
    transform.append(T.Resize(image_size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    dataset = ImageFolder(image_dir, transform)

    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
    return data_loader

Model solver for training the StarGAN, the training parmaeter are the following:
* dimension of domain labels: 5
* image resolution: 224
* number of conv filters in the first layer of G: 64
* number of conv filters in the first layer of D: 64
* number of residual blocks in G: 6
* number of strided conv layers in D: 6
* weight for domain classification loss: 1
* weight for reconstruction loss: 10
* weight for gradient penalty: 10
* mini-batch size: 16
* number of total iterations for training D: 200000
* number of iterations for decaying lr: 100000
* learning rate for G: g_lr
* learning rate for D: d_lr
* number of D updates per each G update: 5
* beta1 for Adam optimizer: 0.5
* beta2 for Adam optimizer: 0.999

In [7]:
class Solver(object):
    """Solver for training StarGAN."""

    def __init__(self, rafd_loader = None, resume_iters = None, g_lr = 0.0001, d_lr = 0.0001):
        """Initialize configurations."""

        # Data loader.
        self.rafd_loader = rafd_loader

        # Model configurations.
        self.c_dim = 5
        self.image_size = 224
        self.g_conv_dim = 64
        self.d_conv_dim = 64
        self.g_repeat_num = 6
        self.d_repeat_num = 6
        self.lambda_cls = 1
        self.lambda_rec = 10
        self.lambda_gp = 10

        # Training configurations.
        self.batch_size = 16
        self.num_iters = 200000 #
        self.num_iters_decay = 100000
        self.g_lr = g_lr
        self.d_lr = d_lr
        self.n_critic = 5
        self.beta1 = 0.5
        self.beta2 = 0.999
        self.resume_iters = resume_iters
        self.selected_attrs = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

        # Miscellaneous.
        self.use_tensorboard = 'True'
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Directories.
        self.log_dir = '/content/drive/MyDrive/Digital/stargan/logs'
        self.sample_dir = '/content/drive/MyDrive/Digital/stargan/samples'
        self.model_save_dir = '/content/drive/MyDrive/Digital/stargan/models'
        self.result_dir = '/content/drive/MyDrive/Digital/stargan/results'

        # Step size.
        self.log_step = 10
        self.sample_step = 1000 #
        self.model_save_step = 2500 #
        self.lr_update_step = 1000

        # Build the model and tensorboard.
        self.build_model()
        if self.use_tensorboard:
            self.build_tensorboard()

    def build_model(self):
        """Create a generator and a discriminator."""
        self.G = Generator(self.g_conv_dim, self.c_dim, self.g_repeat_num)
        self.D = Discriminator(self.image_size, self.d_conv_dim, self.c_dim, self.d_repeat_num)

        self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
        self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])
        self.print_network(self.G, 'G')
        self.print_network(self.D, 'D')

        self.G.to(self.device)
        self.D.to(self.device)

    def print_network(self, model, name):
        """Print out the network information."""
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(model)
        print(name)
        print("The number of parameters: {}".format(num_params))

    def restore_model(self, resume_iters):
        """Restore the trained generator and discriminator."""
        print('Loading the trained models from step {}...'.format(resume_iters))
        G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(resume_iters))
        D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(resume_iters))
        self.G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
        self.D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

    def build_tensorboard(self):
        """Build a tensorboard logger."""
        self.logger = Logger(self.log_dir)

    def update_lr(self, g_lr, d_lr):
        """Decay learning rates of the generator and discriminator."""
        for param_group in self.g_optimizer.param_groups:
            param_group['lr'] = g_lr
        for param_group in self.d_optimizer.param_groups:
            param_group['lr'] = d_lr

    def reset_grad(self):
        """Reset the gradient buffers."""
        self.g_optimizer.zero_grad()
        self.d_optimizer.zero_grad()

    def denorm(self, x):
        """Convert the range from [-1, 1] to [0, 1]."""
        out = (x + 1) / 2
        return out.clamp_(0, 1)

    def gradient_penalty(self, y, x):
        """Compute gradient penalty: (L2_norm(dy/dx) - 1)**2."""
        weight = torch.ones(y.size()).to(self.device)
        dydx = torch.autograd.grad(outputs=y,
                                   inputs=x,
                                   grad_outputs=weight,
                                   retain_graph=True,
                                   create_graph=True,
                                   only_inputs=True)[0]

        dydx = dydx.view(dydx.size(0), -1)
        dydx_l2norm = torch.sqrt(torch.sum(dydx**2, dim=1))
        return torch.mean((dydx_l2norm-1)**2)

    def label2onehot(self, labels, dim):
        """Convert label indices to one-hot vectors."""
        batch_size = labels.size(0)
        out = torch.zeros(batch_size, dim)
        out[np.arange(batch_size), labels.long()] = 1
        return out

    def create_labels(self, c_org, c_dim=5, selected_attrs=None):
        """Generate target domain labels for debugging and testing."""
        c_trg_list = []
        for i in range(c_dim):
            c_trg = self.label2onehot(torch.ones(c_org.size(0))*i, c_dim)

            c_trg_list.append(c_trg.to(self.device))
        return c_trg_list

    def classification_loss(self, logit, target):
        """Compute binary or softmax cross entropy loss."""
        return F.cross_entropy(logit, target)

    def train(self):
        """Train StarGAN within a single dataset."""
        # Set data loader.
        data_loader = self.rafd_loader

        # Fetch fixed inputs for debugging.
        data_iter = iter(data_loader)
        x_fixed, c_org = next(data_iter)
        x_fixed = x_fixed.to(self.device)
        c_fixed_list = self.create_labels(c_org, self.c_dim, self.selected_attrs)

        # Learning rate cache for decaying.
        g_lr = self.g_lr
        d_lr = self.d_lr
        #print(g_lr)
        #print(d_lr)

        # Start training from scratch or resume training.
        start_iters = 0
        if self.resume_iters:
            print('resuming')
            start_iters = self.resume_iters
            self.restore_model(self.resume_iters)

        # Start training.
        print('Start training...')
        start_time = time.time()
        for i in range(start_iters, self.num_iters):

            # =================================================================================== #
            #                             1. Preprocess input data                                #
            # =================================================================================== #

            # Fetch real images and labels.
            try:
                x_real, label_org = next(data_iter)
            except:
                data_iter = iter(data_loader)
                x_real, label_org = next(data_iter)

            # Generate target domain labels randomly.
            rand_idx = torch.randperm(label_org.size(0))
            label_trg = label_org[rand_idx]

            c_org = self.label2onehot(label_org, self.c_dim)
            c_trg = self.label2onehot(label_trg, self.c_dim)

            x_real = x_real.to(self.device)           # Input images.
            c_org = c_org.to(self.device)             # Original domain labels.
            c_trg = c_trg.to(self.device)             # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            # =================================================================================== #
            #                             2. Train the discriminator                              #
            # =================================================================================== #

            # Compute loss with real images.
            out_src, out_cls = self.D(x_real)
            d_loss_real = - torch.mean(out_src)
            d_loss_cls = self.classification_loss(out_cls, label_org)

            # Compute loss with fake images.
            x_fake = self.G(x_real, c_trg)
            out_src, out_cls = self.D(x_fake.detach())
            d_loss_fake = torch.mean(out_src)

            # Compute loss for gradient penalty.
            alpha = torch.rand(x_real.size(0), 1, 1, 1).to(self.device)
            x_hat = (alpha * x_real.data + (1 - alpha) * x_fake.data).requires_grad_(True)
            out_src, _ = self.D(x_hat)
            d_loss_gp = self.gradient_penalty(out_src, x_hat)

            # Backward and optimize.
            d_loss = d_loss_real + d_loss_fake + self.lambda_cls * d_loss_cls + self.lambda_gp * d_loss_gp
            self.reset_grad()
            d_loss.backward()
            self.d_optimizer.step()

            # Logging.
            loss = {}
            loss['D/loss_real'] = d_loss_real.item()
            loss['D/loss_fake'] = d_loss_fake.item()
            loss['D/loss_cls'] = d_loss_cls.item()
            loss['D/loss_gp'] = d_loss_gp.item()

            # =================================================================================== #
            #                               3. Train the generator                                #
            # =================================================================================== #

            if (i+1) % self.n_critic == 0:
                # Original-to-target domain.
                x_fake = self.G(x_real, c_trg)
                out_src, out_cls = self.D(x_fake)
                g_loss_fake = - torch.mean(out_src)
                g_loss_cls = self.classification_loss(out_cls, label_trg)

                # Target-to-original domain.
                x_reconst = self.G(x_fake, c_org)
                g_loss_rec = torch.mean(torch.abs(x_real - x_reconst))

                # Backward and optimize.
                g_loss = g_loss_fake + self.lambda_rec * g_loss_rec + self.lambda_cls * g_loss_cls
                self.reset_grad()
                g_loss.backward()
                self.g_optimizer.step()

                # Logging.
                loss['G/loss_fake'] = g_loss_fake.item()
                loss['G/loss_rec'] = g_loss_rec.item()
                loss['G/loss_cls'] = g_loss_cls.item()

            # =================================================================================== #
            #                                 4. Miscellaneous                                    #
            # =================================================================================== #

            # Print out training information.
            if (i+1) % self.log_step == 0:
                et = time.time() - start_time
                et = str(datetime.timedelta(seconds=et))[:-7]
                log = "Elapsed [{}], Iteration [{}/{}]".format(et, i+1, self.num_iters)
                for tag, value in loss.items():
                    log += ", {}: {:.4f}".format(tag, value)
                print(log)

                if self.use_tensorboard:
                    for tag, value in loss.items():
                        self.logger.scalar_summary(tag, value, i+1)

            # Translate fixed images for debugging.
            if (i+1) % self.sample_step == 0:
                with torch.no_grad():
                    x_fake_list = [x_fixed]
                    for c_fixed in c_fixed_list:
                        x_fake_list.append(self.G(x_fixed, c_fixed))
                    x_concat = torch.cat(x_fake_list, dim=3)
                    sample_path = os.path.join(self.sample_dir, '{}-images.jpg'.format(i+1))
                    save_image(self.denorm(x_concat.data.cpu()), sample_path, nrow=1, padding=0)
                    print('Saved real and fake images into {}...'.format(sample_path))

            # Save model checkpoints.
            if (i+1) % self.model_save_step == 0:
                G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(i+1))
                D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(i+1))
                torch.save(self.G.state_dict(), G_path)
                torch.save(self.D.state_dict(), D_path)
                print('Saved model checkpoints into {}...'.format(self.model_save_dir))

            # Decay learning rates.
            if (i+1) % self.lr_update_step == 0 and (i+1) > (self.num_iters - self.num_iters_decay):
                g_lr -= (self.g_lr / float(self.num_iters_decay))
                d_lr -= (self.d_lr / float(self.num_iters_decay))
                self.update_lr(g_lr, d_lr)
                print ('Decayed learning rates, g_lr: {}, d_lr: {}.'.format(g_lr, d_lr))



## Train

Load the trainingg data and the solver

In [ ]:
rafd_loader = get_loader('/content/Data/dataset_quad', None, None, 224, 224, 16, 'train', 1)


solver = Solver(rafd_loader)

The number of parameters: 8430528
The number of parameters: 44813248


Actual model train

In [ ]:
solver.train()

## Test

Extract test data

In [ ]:
!unzip "/content/drive/MyDrive/Digital/Data/test_prova 2.zip" -d Test5

Archive:  /content/drive/MyDrive/Digital/Data/test_prova 2.zip
   creating: Test5/test_prova 2/
  inflating: Test5/__MACOSX/._test_prova 2  
   creating: Test5/test_prova 2/Happy/
  inflating: Test5/test_prova 2/.DS_Store  
  inflating: Test5/__MACOSX/test_prova 2/._.DS_Store  
 extracting: Test5/test_prova 2/Icon  
  inflating: Test5/__MACOSX/test_prova 2/._Icon  
   creating: Test5/test_prova 2/Sad/
   creating: Test5/test_prova 2/Surprise/
   creating: Test5/test_prova 2/Neutral/
   creating: Test5/test_prova 2/Angry/
  inflating: Test5/test_prova 2/Happy/WhatsApp-Image-2024-02-12-at-15.57.40-_1_.png  
  inflating: Test5/__MACOSX/test_prova 2/Happy/._WhatsApp-Image-2024-02-12-at-15.57.40-_1_.png  
  inflating: Test5/test_prova 2/Happy/.DS_Store  
  inflating: Test5/__MACOSX/test_prova 2/Happy/._.DS_Store  
  inflating: Test5/test_prova 2/Happy/WhatsApp-Image-2024-02-12-at-15.57.39.png  
  inflating: Test5/__MACOSX/test_prova 2/Happy/._WhatsApp-Image-2024-02-12-at-15.57.39.png  
  in

Resize the test images

In [ ]:
input_directory = "/content/Test5/test_prova 2"
output_directory = "/content/Test5/test_quad 2"
process_images_in_directory(input_directory, output_directory)

  0%|          | 0/4 [00:00<?, ?it/s]<ipython-input-5-2eab3d3d7807>:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(output_size, Image.ANTIALIAS)
100%|██████████| 7/7 [00:00<00:00, 40.95it/s]


Load the test data

In [ ]:
prova_loader = get_loader('/content/Test5/test_quad 2', None, None, 224, 224, 16, 'test', 1)

Call the trained solver

In [ ]:
solver = Solver(prova_loader, resume_iters = 200000)

Produce the trasformed test images in the destination folder

In [ ]:
solver.restore_model(200000)

data_loader = prova_loader

with torch.no_grad():
  for i, (x_real, c_org) in enumerate(data_loader):
    # Prepare input images and target domain labels.
    x_real = x_real.to(solver.device)
    c_trg_list = solver.create_labels(c_org, solver.c_dim, solver.selected_attrs)

    # Translate images.
    x_fake_list = [x_real]
    for c_trg in c_trg_list:
      x_fake_list.append(solver.G(x_real, c_trg))

    # Save the translated images.
    x_concat = torch.cat(x_fake_list, dim=3)
    result_path = os.path.join(solver.result_dir, '{}-images.jpg'.format(i+1))
    save_image(solver.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
    print('Saved real and fake images into {}...'.format(result_path))

Loading the trained models from step 200000...
Saved real and fake images into /content/drive/MyDrive/Digital/stargan/results/1-images.jpg...
Saved real and fake images into /content/drive/MyDrive/Digital/stargan/results/2-images.jpg...
